In [ ]:
def find_seed(g, x=0, c =0, eps = 2**(-26)):
    """
    trouve la valeur t telle que g(t) = c à epsilon près
    """
    
    gauche = 0
    droite = 1
    
    if (g(x,0)-c)*(g(x,1)-c) > 0 :
        return None
        
    while ((droite-gauche)/2) >= eps :
        milieu = (droite + gauche)/2
        
        if (g(x,droite)-c)*(g(x,milieu)-c)<=0 :
            gauche = milieu
        
        else :
            droite = milieu
    
    return (droite+gauche)/2